In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
from sympy import Symbol
from sympy.solvers import solve
sm.init_printing(use_unicode=True) # for pretty printing
from IPython.display import display
import matplotlib.pyplot as plt # baseline modul
import ipywidgets as widgets


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
#import modelproject


# The Solow model with human capital

## Variables 

* $Y_t$  : Production
* $L_t$ : Labor
* $A_t$ : Technology
* $K_t$ : Phisical Capital
* $H_t$ : Human Capital

## Parameters

* $s_h$:       Savings/Investments in human capital\
* $s_k$:       Savings/Investments in physical capital\
* $g$:         Growth rate of technology\
* $n$:         Growth rate of labour force
* $\delta$:     Deprisiation rate of physical capital\
* $\alpha$:     Return physical capital\
* $\phi$:       Return on human capital\
* $h$:         Human capital per effective worker (Stock)\

## Equations 

**Production funciton** 

$ Y_{t}=K_{t}^{\alpha}H_{t}^{\varphi}(A_{t}L_{t})^{1-\alpha-\varphi} $

**Saving** \
Constant fraction of production in the eoconomy

$ S_t = (s_H+s_K)Y_t,\, s_H+s_K\in(0,1) $

**Capital Accumulation**

$ K_{t+1}=s_{K}Y_t+(1-\delta)K_{t}$

**Human capital accumulation** 

$ H_{t+1} = s_{H}Y_t+(1-\delta)H_t $

**Population and teknology (TFP) growth**

$ L_{t+1} = (1+n)L_t $

$ A_{t+1} = (1+g)A_t $

## Tilde equations

Per effectiv worker 


**Defitiontions**

Production, fysical capital and human capital as divided by popluation and teknology to get .... \
$   \tilde{y}_{t} = \frac{Y_{t}} {A_tL_t} $\
$   \tilde{k}_{t} = \frac{K_{t}} {A_tL_t} $\
$   \tilde{h}_{t} = \frac{H_{t}} {A_tL_t} $

**Transition Equations** 

The tilde transition eqautions can be found by inserting the tilde definitions ...

$ \tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[s_K\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{k}_{t}] $

$ \tilde{h}_{t+1}= \frac{1}{(1+n)(1+g)}[s_H\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{h}_{t}] $




# Analytical solution

1. Defining the variables as symbols for using sympy

In [2]:

k = sm.symbols('k')
h = sm.symbols('h')
y = sm.symbols('y')
c = sm.symbols('c')
sh = sm.symbols('s_h')
sk = sm.symbols('s_k')
delta = sm.symbols('delta')
phi = sm.symbols('phi')
alpha = sm.symbols('alpha')
g = sm.symbols('g')
n = sm.symbols('n')

x = [sk, sh, g, phi, n, alpha, delta]


2. Function for deriving the analytical solution

In [3]:
# Analitical soluction - findes the solow equations and return the nult line 
def analytical(x, latex = False):

    """ Derives the equations for the nultlines based on the transition equations for human and fysical capital

    Args: 
    x:  list of parameters

    Returns:
    Nullclines for physical and human capital h(k)

    """

    #Transition lines:
    k_t2 = (sk*k**alpha*h**phi+(1-delta)*k)/((1+n)*(1+g))
    h_t2 = (sh*k**alpha*h**phi+(1-delta)*h)/((1+n)*(1+g))

    #Change in capital stock () - Sollow equations 
    delta_k = k_t2 - k 
    delta_h = h_t2 - h
    delta_combined = delta_h + delta_k

    #Nultlines (Stock of capital is constant)
    sollow_k = sm.Eq(delta_k,0)
    sollow_h = sm.Eq(delta_h,0)

    #Isolating k, and h in each equation - For drawing the phase diagram h should be isolated in both
    nult_k = sm.solve(sollow_k,h)
    nult_h = sm.solve(sollow_h,h)

    if latex == True: 

        print(f'  Latex code for the nult line for physical capital: \n {sm.latex(nult_k)} \n\n')
        print(f'  Latex code for the nult line for human capital: \n  {sm.latex(nult_h)}')



    return nult_k, nult_h, delta_combined
    #return h_nult_vek, k_nult_vek


In [4]:
nult_k, nult_h, delta_combined = analytical(x, False)


In [5]:
delta_combined

                      φ  α       φ  α               
         h⋅(1 - δ) + h ⋅k ⋅sₕ   h ⋅k ⋅sₖ + k⋅(1 - δ)
-h - k + ──────────────────── + ────────────────────
           (g + 1)⋅(n + 1)        (g + 1)⋅(n + 1)   

In [6]:
nult_h, nult_k

⎛⎡                         1  ⎤                                   ⎞
⎜⎢                       ─────⎥                                   ⎟
⎜⎢                       φ - 1⎥  ⎡     __________________________⎤⎟
⎜⎢⎛ -α                  ⎞     ⎥  ⎢    ╱  1 - α                   ⎥⎟
⎜⎢⎜k  ⋅(δ + g⋅n + g + n)⎟     ⎥  ⎢   ╱  k     ⋅(δ + g⋅n + g + n) ⎥⎟
⎜⎢⎜─────────────────────⎟     ⎥, ⎢φ ╱   ──────────────────────── ⎥⎟
⎝⎣⎝          sₕ         ⎠     ⎦  ⎣╲╱               sₖ            ⎦⎠



Nulcline for pysical capital:

$$ \tilde{h_t} = \left[ \left(\frac{k^{1 - \alpha} \left(\delta + g n + g + n\right)}{s_{k}}\right)^{\frac{1}{\phi}}\right], \quad \Delta \tilde{k}_t = 0 $$



Nulcline for pysical capital:

$$ \tilde{h_t} =\left[ \left(\frac{k^{- \alpha} \left(\delta + g n + g + n\right)}{s_{h}}\right)^{\frac{1}{\phi - 1}}\right], \quad \Delta \tilde{h}_t = 0$$

# Stop her

In [ ]:
nult_h - nult_k

#sollow_h = sm.Eq(nult_k - nult_h,0)

In [ ]:
def analytical(x):
    #Transition lines:
    k_t2 = (sk*k**alpha*h**phi+(1-delta)*k)/((1+n)*(1+g))
    h_t2 = (sh*k**alpha*h**phi+(1-delta)*h)/((1+n)*(1+g))

    #Change in capital stock () - Sollow equations 
    delta_k = k_t2 - k 
    delta_h = h_t2 - h

    #Nultlines (Stock of capital is constant)
    sollow_k = sm.Eq(delta_k,0)
    sollow_h = sm.Eq(delta_h,0)

    #Isolating k, and h in each equation - For drawing the phase diagram h should be isolated in both
    nult_k = sm.solve(sollow_k,h)
    nult_h = sm.solve(sollow_h,h)[2]

    #nult_h_f = sm.lambdify((k), nult_h)[1]
    #nult_k_f = sm.lambdify((k), nult_k)


    return nult_k, nult_h
    #return h_nult_vek, k_nult_vek


In [ ]:
nult_k, nult_h = analytical(x)

In [ ]:
nult_k, nult_h

In [ ]:
#Numeric med den over

nult_h_f = sm.lambdify((k), nult_h)
nult_k_f = sm.lambdify((k), nult_k)


h_nult_vek =  nult_h_f(k_vek)[2]
k_nult_vek =  nult_k_f(k_vek)[0]

In [ ]:
out = test(x, k_vek)

h_nult_vek = out[0]
k_nult_vek = out[1]

In [ ]:
def numeric(x):
    #Transition lines:
    k_t2 = (sk*k**alpha*h**phi+(1-delta)*k)/((1+n)*(1+g))
    h_t2 = (sh*k**alpha*h**phi+(1-delta)*h)/((1+n)*(1+g))

    #Change in capital stock () - Sollow equations 
    delta_k = k_t2 - k 
    delta_h = h_t2 - h

    #Nultlines (Stock of capital is constant)
    sollow_k = sm.Eq(delta_k,0)
    sollow_h = sm.Eq(delta_h,0)

    #Isolating k, and h in each equation - For drawing the phase diagram h should be isolated in both
    nult_k = sm.solve(sollow_k,h)
    nult_h = sm.solve(sollow_h,h)

    nult_h_f = sm.lambdify((k), nult_h)
    nult_k_f = sm.lambdify((k), nult_k)


    #h_nult_vek =  nult_h_f(k_vek)[2]
    #k_nult_vek =  nult_k_f(k_vek)[0]

    return delta_k
    #return h_nult_vek, k_nult_vek


In [ ]:
nult_h_f = sm.lambdify((k), nult_h)
nult_k_f = sm.lambdify((k), nult_k)